<a href="https://colab.research.google.com/github/carlm451/Gemstone_Images_Classification_Fine_Tuning/blob/main/Gemstones_Classifier_MobileNet_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import numpy as np

import os
import matplotlib.pyplot as plt
from random import shuffle

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# Download/Prepare Kaggle Gemstones Dataset

need to have an API key from kaggle, follow instructions here https://www.kaggle.com/docs/api

In [2]:
# pulling gemstones data from kaggle
#!pip install kaggle

!mkdir ~/.kaggle

#need a kaggle API key kaggle.json
# I just upload it from my pc to /content colab directory 
!mv kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

#download the images to local colab drive
!kaggle datasets download lsind18/gemstones-images

# unzip the images
!unzip gemstones-images.zip &> /dev/null  #suppress terminal output when unzipping images

#expect to zee gemstones-images.zip, and test and train directories 
!ls 

 81% 45.0M/55.2M [00:00<00:00, 157MB/s]
100% 55.2M/55.2M [00:00<00:00, 159MB/s]
gemstones-images.zip  sample_data  test  train


In [3]:
import os

data_dir = '/content'

train_dir = os.path.join(data_dir,'train')

!mkdir val # going to use some training data for validation, save test data for final model evaluation 
val_dir = os.path.join(data_dir,'val')

test_dir = os.path.join(data_dir,'test')

def count_img_samples(directory):
    
    count = 0
    
    for i,gem_type in enumerate(os.listdir(directory)):
        
        gem_dir = os.path.join(directory,gem_type)
    
        img_list = os.listdir(gem_dir)

        #print(f' dir {gem_dir} has {len(img_list)} images')

        count += len(img_list)
    
    return count

n_train = count_img_samples(train_dir)
n_test = count_img_samples(test_dir)
n_val = count_img_samples(val_dir)

print(f'{n_train=}, {n_val=}, {n_test=}')

n_train=2856, n_val=0, n_test=363


In [4]:
from random import shuffle

def partition_val_data(train_dir,val_dir,val_split=0.1):
    
    for gem_type in os.listdir(train_dir):
        
        train_gem_dir = os.path.join(train_dir,gem_type)
        
        img_list = os.listdir(train_gem_dir)
        
        shuffle(img_list)
        
        n_samples = round(len(img_list)*val_split)
        
        val_img_list = img_list[:n_samples] # take n_samples random images to move
        
        val_gem_dir = os.path.join(val_dir,gem_type)
        
        if not os.path.exists(val_gem_dir):
            
            os.mkdir(val_gem_dir)
            
            for gem_img in val_img_list:
                
                original_path = os.path.join(train_gem_dir,gem_img)
                
                destination_path = os.path.join(val_gem_dir,gem_img)
                
                os.rename(original_path,destination_path)
        
            #print(f'Moved {len(os.listdir(val_gem_dir))} training images from to {val_gem_dir}')
            
        else:
            
            pass
            #print(f'Val dir {val_gem_dir} has {len(os.listdir(val_gem_dir))} images')

In [5]:
val_split=0.15  # move 15% train to use for validation

partition_val_data(train_dir,val_dir,val_split)

n_train = count_img_samples(train_dir)
n_test = count_img_samples(test_dir)
n_val = count_img_samples(val_dir)

print(f'{n_train=}, {n_val=}, {n_test=}')

n_train=2434, n_val=422, n_test=363


In [6]:
!ls val

gem_types_list = os.listdir(val_dir)

n_classes = len(gem_types_list)

print(f'{n_classes} classes of gemstone')

 Alexandrite	      Chrysocolla     Larimar		 'Sapphire Blue'
 Almandine	      Chrysoprase     Malachite		 'Sapphire Pink'
 Amazonite	      Citrine	      Moonstone		 'Sapphire Purple'
 Amber		      Coral	      Morganite		 'Sapphire Yellow'
 Amethyst	      Danburite      'Onyx Black'	  Scapolite
 Ametrine	      Diamond	     'Onyx Green'	  Serpentine
 Andalusite	      Diaspore	     'Onyx Red'		  Sodalite
 Andradite	      Dumortierite    Opal		  Spessartite
 Aquamarine	      Emerald	      Pearl		  Sphene
'Aventurine Green'    Fluorite	      Peridot		  Spinel
'Aventurine Yellow'  'Garnet Red'     Prehnite		  Spodumene
 Benitoite	      Goshenite       Pyrite		  Sunstone
'Beryl Golden'	      Grossular       Pyrope		  Tanzanite
 Bixbite	      Hessonite      'Quartz Beer'	 'Tigers Eye'
 Bloodstone	      Hiddenite      'Quartz Lemon'	  Topaz
'Blue Lace Agate'     Iolite	     'Quartz Rose'	  Tourmaline
 Carnelian	      Jade	     'Quartz Rutilated'   Tsavorite
'Cats Eye'	      Jasper	     'Quartz 

In [7]:
# generators to stream images for training/validation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1.0/255.,
                                   rotation_range=90,
                                   width_shift_range=0.4,
                                   height_shift_range=0.4,
                                   zoom_range=0.5,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  )

val_datagen  = ImageDataGenerator( rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    target_size=(224, 224),
                                                    keep_aspect_ratio=False,
                                                    classes=gem_types_list) 

val_generator = val_datagen.flow_from_directory(val_dir,
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    target_size=(224,224),
                                                    keep_aspect_ratio=False,
                                                    classes=gem_types_list)

Found 2434 images belonging to 87 classes.
Found 422 images belonging to 87 classes.


# Classifier based on pretrained ResNet50 model

In [8]:
def get_uncompiled_model(n_classes, model_name,fine_tune=0):

    tf.keras.backend.clear_session()

    pretrained = MobileNetV2(include_top=False,pooling='avg',input_shape=(224,224,3))

    if fine_tune > 0:
        for layer in pretrained.layers[:-fine_tune]:
            layer.trainable = False
    else:
        pretrained.trainable=False #freezes all children layers 

    inputs=tf.keras.layers.Input(shape=(224,224,3))

    x=pretrained(inputs,training=False)

    x = tf.keras.layers.Dense(n_classes)(x)  # make sure to use from_logits=True in loss later on

    model = tf.keras.Model(inputs=inputs,outputs=x)
    
    return model

In [12]:
model_1 = get_uncompiled_model(n_classes,model_name='mobilenetV2_frozen')
    
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 dense (Dense)               (None, 87)                111447    
                                                                 
Total params: 2,369,431
Trainable params: 111,447
Non-trainable params: 2,257,984
_________________________________________________________________


In [13]:

base_learning_rate = 0.0005

model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics = [tf.keras.metrics.CategoricalAccuracy()])

In [14]:
def scheduler(epoch, lr):
    return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

checkpoint_path = "training_1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

N_EPOCHS=25

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 verbose=1,
                                                 save_freq='epoch')

history_1 = model_1.fit(
            train_generator,
            epochs=N_EPOCHS,
            validation_data=val_generator,
            verbose=1,
            callbacks=[cp_callback,lr_callback]
            )

Epoch 1/25
39/39 [==============================] - ETA: 0s - loss: 4.2818 - categorical_accuracy: 0.0674
Epoch 1: val_loss improved from inf to 3.74149, saving model to training_1/cp-0001.ckpt
39/39 [==============================] - 40s 954ms/step - loss: 4.2818 - categorical_accuracy: 0.0674 - val_loss: 3.7415 - val_categorical_accuracy: 0.1540 - lr: 5.0000e-04
Epoch 2/25
39/39 [==============================] - ETA: 0s - loss: 3.2743 - categorical_accuracy: 0.2551
Epoch 2: val_loss improved from 3.74149 to 3.06195, saving model to training_1/cp-0002.ckpt
39/39 [==============================] - 35s 882ms/step - loss: 3.2743 - categorical_accuracy: 0.2551 - val_loss: 3.0620 - val_categorical_accuracy: 0.2701 - lr: 5.0000e-04
Epoch 3/25
39/39 [==============================] - ETA: 0s - loss: 2.7161 - categorical_accuracy: 0.3669
Epoch 3: val_loss improved from 3.06195 to 2.65503, saving model to training_1/cp-0003.ckpt
39/39 [==============================] - 35s 908ms/step - loss: 

In [15]:
epoch=24
last = os.path.join('training_1',f'cp-{epoch:04d}.ckpt')
last

'training_1/cp-0024.ckpt'

In [22]:
model_2 = get_uncompiled_model(n_classes,model_name='mobilenetV2_finetune',fine_tune=4)
    
model_2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 dense (Dense)               (None, 87)                111447    
                                                                 
Total params: 2,369,431
Trainable params: 523,607
Non-trainable params: 1,845,824
_________________________________________________________________


In [23]:
#start from checkpoint epoch 10

model_2.load_weights(last)

fine_tune_lr = base_learning_rate

model_2.compile(optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=fine_tune_lr,momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics = [tf.keras.metrics.CategoricalAccuracy()])

In [24]:
def scheduler(epoch, lr):
    return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

N_EPOCHS=25

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor='val_categorical_accuracy',
                                                 verbose=1,
                                                 save_freq='epoch')

history_2 = model_2.fit(
            train_generator,
            epochs=N_EPOCHS,
            validation_data=val_generator,
            verbose=1,
            callbacks=[cp_callback,lr_callback]
            )

Epoch 1/25
39/39 [==============================] - ETA: 0s - loss: 0.9404 - categorical_accuracy: 0.7572
Epoch 1: val_categorical_accuracy improved from -inf to 0.63507, saving model to training_2/cp-0001.ckpt
39/39 [==============================] - 39s 941ms/step - loss: 0.9404 - categorical_accuracy: 0.7572 - val_loss: 1.3732 - val_categorical_accuracy: 0.6351 - lr: 5.0000e-04
Epoch 2/25
39/39 [==============================] - ETA: 0s - loss: 0.9072 - categorical_accuracy: 0.7592
Epoch 2: val_categorical_accuracy improved from 0.63507 to 0.63981, saving model to training_2/cp-0002.ckpt
39/39 [==============================] - 35s 898ms/step - loss: 0.9072 - categorical_accuracy: 0.7592 - val_loss: 1.3484 - val_categorical_accuracy: 0.6398 - lr: 5.0000e-04
Epoch 3/25
39/39 [==============================] - ETA: 0s - loss: 0.8810 - categorical_accuracy: 0.7560
Epoch 3: val_categorical_accuracy improved from 0.63981 to 0.65640, saving model to training_2/cp-0003.ckpt
39/39 [========